# Projeto Final de Machine Learning

<strong>Membros:</strong>
- João Laet
- Nicolas Cho
- Roger Pina

<strong>Professor:</strong>
- Fábio Ayres

<strong>Github:</strong>
- https://github.com/NicolasCho/projeto_ML

# Imports

In [185]:
import math
import pandas as pd
#from keras import tensorflow

# Data Exploration

## Data Section 1 - The basics

In [4]:
basic_path = './mens-machine-learning-competition-2018/DataFiles'

In [5]:
df_teams = pd.read_csv(basic_path + '/Teams.csv')
seasons = pd.read_csv(basic_path + '/Seasons.csv')
NCAATourneySeeds = pd.read_csv(basic_path + '/NCAATourneySeeds.csv')
RegularSeasonCompactResults = pd.read_csv(basic_path + '/RegularSeasonCompactResults.csv')
NCAATourneyCompactResults = pd.read_csv(basic_path + '/NCAATourneyCompactResults.csv')
SampleSubmissionStage1 = pd.read_csv('./mens-machine-learning-competition-2018/SampleSubmissionStage1.csv')

### Teams

In [6]:
df_teams

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018
...,...,...,...,...
359,1460,Wright St,1988,2018
360,1461,Wyoming,1985,2018
361,1462,Xavier,1985,2018
362,1463,Yale,1985,2018


### SeasonsEloRating

In [7]:
seasons

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast
5,1990,10/30/1989,East,Midwest,Southeast,West
6,1991,10/29/1990,East,Southeast,Midwest,West
7,1992,11/4/1991,East,West,Midwest,Southeast
8,1993,11/2/1992,East,Midwest,Southeast,West
9,1994,11/1/1993,East,Southeast,Midwest,West


### Seeds

In [8]:
NCAATourneySeeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2145,2017,Z12,1292
2146,2017,Z13,1457
2147,2017,Z14,1245
2148,2017,Z15,1297


In [378]:
NCAATourneySeeds = NCAATourneySeeds[NCAATourneySeeds['Season'] < 2014]

### RegularSeasonCompactResults

In [56]:
RegularSeasonCompactResults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
150679,2017,132,1276,71,1458,56,N,0
150680,2017,132,1343,71,1463,59,N,0
150681,2017,132,1348,70,1433,63,N,0
150682,2017,132,1374,71,1153,56,N,0


In [376]:
RegularSeasonCompactResults = RegularSeasonCompactResults[RegularSeasonCompactResults['Season'] < 2014]

### NCAATourneyCompactResults

In [10]:
NCAATourneyCompactResults 

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2112,2017,146,1314,75,1246,73,N,0
2113,2017,146,1376,77,1196,70,N,0
2114,2017,152,1211,77,1376,73,N,0
2115,2017,152,1314,77,1332,76,N,0


In [379]:
NCAATourneyCompactResults = NCAATourneyCompactResults[NCAATourneyCompactResults['Season'] < 2014]

### SampleSubmissionStage1

In [11]:
SampleSubmissionStage1

,ID,Pred
0,2014_1107_1110,0.5
1,2014_1107_1112,0.5
2,2014_1107_1113,0.5
3,2014_1107_1124,0.5
4,2014_1107_1140,0.5
...,...,...
9107,2017_1455_1458,0.5
9108,2017_1455_1462,0.5
9109,2017_1457_1458,0.5
9110,2017_1457_1462,0.5


In [12]:
#pbp2010 = pd.read_csv('./mens-machine-learning-competition-2018/PlayByPlay_2010/Events_2010.csv')

# Feature Engineering

## Elo Rating

In [380]:
def win_probs(wTeam_elo, lTeam_elo, home_court_advantage, WLoc):
    w = math.pow(10, wTeam_elo/400)
    l = math.pow(10, lTeam_elo/400)
    a = math.pow(10, home_court_advantage/400) 
    
    if WLoc == 'H':
        denom = l + a*w
        wTeam_prob = a*w / denom
        lTeam_prob = l / denom   
        
    elif WLoc == 'A':
        denom = a*l + w
        wTeam_prob = w / denom
        lTeam_prob = a*l / denom   
        
    else:
        denom = l + w
        wTeam_prob = a / denom
        lTeam_prob = l / denom 
        
    return wTeam_prob, lTeam_prob

In [381]:
def elo_k(MOV, elo_diff):
    k = 20
    if MOV>0:
        multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
    else:
        multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
    return k*multiplier

In [382]:
def update_elo(game, df_R):
    home_court_advantage = 100
    wTeam = game['WTeamID']
    wTeam_elo = int(df_R[df_R['TeamID'] == wTeam]['EloRating_start'])
    wTeam_score = game['WScore']
    #print(wTeam, wTeam_elo, wTeam_score)
    
    lTeam = game['LTeamID']
    lTeam_elo = int(df_R[df_R['TeamID'] == lTeam]['EloRating_start'])
    lTeam_score = game['LScore']
    #print(lTeam, lTeam_elo, lTeam_score)
        
    wTeam_prob, lTeam_prob = win_probs(wTeam_elo, lTeam_elo, home_court_advantage, game['WLoc'])
    
    k = elo_k(wTeam_score - lTeam_score, wTeam_elo - lTeam_elo)
        
    updated_wTeam_elo = wTeam_elo + k * (1 - wTeam_prob) 
    updated_lTeam_elo = lTeam_elo + k * (0 - lTeam_prob)
    
    return updated_wTeam_elo, updated_lTeam_elo

In [385]:
def dfER(season):
    if season == 1985:
        ER_start = list(None for e in range(len(df_eloRating.index)))
        for e in range(len(df_eloRating[df_eloRating['Season'] == 1985])):
            ER_start[e] = 1500
        df_eloRating['EloRating_start'] = ER_start
        
    else:
        df_lastSeason = df_eloRating[df_eloRating['Season'] == season - 1]
        
        for team in df_eloRating[df_eloRating['Season'] == season]['TeamID']:
            idx = df_eloRating[(df_eloRating['Season'] == season) & (df_eloRating['TeamID'] == team)].index[0]
            if team in list(df_lastSeason['TeamID']):
                df_eloRating.loc[idx, 'EloRating_start'] = int(df_lastSeason[df_lastSeason['TeamID'] == team]['EloRating_end'])
            else:
                df_eloRating.loc[idx, 'EloRating_start'] = 1500


In [386]:
df_eloRating = NCAATourneySeeds.copy()
for season in df_eloRating['Season'].unique(): # para cada season
    
    dfER(season)
    
    df_games = NCAATourneyCompactResults[NCAATourneyCompactResults['Season'] == season] #jogos da season
    
    df_R = df_eloRating[df_eloRating['Season'] == season][['Season','TeamID', 'EloRating_start']]
    
    for e in range(len(df_games.index)):
        game = df_games.iloc[e] 
        
        updated_wTeam_elo, updated_lTeam_elo = update_elo(game, df_R)
       
        #wTeam
        idx = df_R[df_R['TeamID'] == game['WTeamID']].index[0]
        df_R.loc[idx, 'EloRating_start'] = updated_wTeam_elo
        
        #lTeam
        idx = df_R[df_R['TeamID'] == game['LTeamID']].index[0]
        df_R.loc[idx, 'EloRating_start'] = updated_lTeam_elo
        
    
    for e in df_R.index:
        idx = df_eloRating[(df_eloRating['Season'] == season) & (df_eloRating['TeamID'] == df_R.loc[e]['TeamID'])].index[0]
        df_eloRating.loc[idx, 'EloRating_end'] = df_R.loc[e]['EloRating_start']

In [387]:
df_eloRating

,Season,Seed,TeamID,EloRating_start,EloRating_end
0,1985,W01,1207,1500.0,1627.018229
1,1985,W02,1210,1500.0,1555.963459
2,1985,W03,1228,1500.0,1548.482141
3,1985,W04,1260,1500.0,1521.776949
4,1985,W05,1374,1500.0,1515.785522
...,...,...,...,...,...
1873,2013,Z13a,1129,1500.0,1490.266171
1874,2013,Z13b,1247,1500.0,1525.373901
1875,2013,Z14,1217,1489.0,1487.494729
1876,2013,Z15,1233,1493.0,1485.413891


## Performance Recente

In [388]:
NCAATourneyCompactResults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
1844,2013,146,1257,85,1181,63,N,0
1845,2013,146,1276,79,1196,59,N,0
1846,2013,152,1257,72,1455,68,N,0
1847,2013,152,1276,61,1393,56,N,0
